# HunyuanVideo on Google Colab A100

Generate high-quality videos using HunyuanVideo model on Google Colab A100 GPU

**Requirements:**
- Google Colab Pro/Pro+ with A100 GPU (40GB VRAM)
- Runtime: Python 3 with GPU acceleration

In [ ]:
# Check GPU and setup environment
!nvidia-smi

import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Install HunyuanVideo dependencies with stable versions\n!pip install diffusers[torch]==0.33.1\n!pip install transformers==4.52.4\n!pip install accelerate==1.1.1  # Stable version compatible with transformers 4.52.4\n!pip install safetensors==0.4.5  # Stable version from late 2024\n!pip install xformers==0.0.28.post2  # Stable version for CUDA 12.x\n!pip install imageio-ffmpeg  # For video processing

In [ ]:
import torch
import numpy as np
from diffusers import HunyuanVideoPipeline
import imageio
from IPython.display import Video, display
import gc
import os

In [ ]:
# Load HunyuanVideo with A100 optimizations
model_id = "hunyuanvideo-community/HunyuanVideo"

print("Loading HunyuanVideo pipeline...")
pipe = HunyuanVideoPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Memory optimizations for A100 40GB
pipe.enable_sequential_cpu_offload()
pipe.vae.enable_tiling()
pipe.vae.enable_slicing()

print("Model loaded successfully!")

In [ ]:
# Generate video
prompt = "A majestic golden retriever running through a sunlit meadow, slow motion, cinematic"
seed = 42

# Set seed for reproducibility
torch.manual_seed(seed)
np.random.seed(seed)

print(f"Generating: {prompt}")

with torch.inference_mode():
    video_frames = pipe(
        prompt=prompt,
        num_frames=49,
        height=720,
        width=1280,
        num_inference_steps=25,
        guidance_scale=7.0,
    ).frames[0]

print(f"Generated {len(video_frames)} frames")

In [ ]:
# Save and display video
output_path = "hunyuan_video.mp4"
fps = 25

# Convert PIL to numpy arrays
frames_array = [np.array(frame.convert('RGB')) for frame in video_frames]

# Save video
imageio.mimsave(
    output_path,
    frames_array,
    fps=fps,
    codec='libx264',
    quality=8
)

print(f"Video saved: {output_path}")

# Display in notebook
display(Video(output_path, width=640))

In [ ]:
# Clean up memory
del pipe
del video_frames
del frames_array
torch.cuda.empty_cache()
gc.collect()

print("Memory cleanup completed")